In [1]:
from tensorflow.keras.models import load_model
from utility.loss import seg_loss, dice_coef, iou_coef
from sklearn.preprocessing import binarize
import tensorflow as tf
import numpy as np
import os
import glob
os.environ["CUDA_VISIBLE_DEVICES"]="0"

2023-04-30 11:28:04.545289: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def npy_loader(maindir, rtype, seed: int = 2):
    # Get list of files in directory
    directory = maindir + '*.npy'
    pathlist = glob.glob(directory)
    
    # Iterate over list of files
    for path in pathlist:
        array = np.load(path)
        img = array[:, :, 0]
        img = (img * 2) - 1
        if rtype == 'elli':
            mask = array[:, :, 1]
        else:
            mask = binarize(array[:, :, 1])
        yield img[..., np.newaxis], mask[..., np.newaxis]
        
def npy_dataset(maindir, shape_i, shape_m, seed: int = 2, batch: int = 1, rtype='elli'):
    ds = tf.data.Dataset.from_generator(lambda: npy_loader(maindir=maindir, rtype=rtype, seed = seed), 
                                        output_types=(tf.float32, tf.float32), 
                                        output_shapes=(shape_i, shape_m))
    return ds.batch(batch)

### Elliptical ROI Dataset

In [4]:
# load data
batch_size = 1

elli_test_df = npy_dataset('/home/careinfolab/unet_mammo/images/pos_norm_elli/test/', 
                           (1024, 832, 1),
                           (1024, 832, 1),
                           seed=2, 
                           batch=batch_size, 
                           rtype='elli')

### Rectangular ROI Dataset

In [3]:
# load data
batch_size = 1

rect_test_df = npy_dataset('/home/careinfolab/unet_mammo/images/pos_norm/test/', 
                           (1024, 832, 1),
                           (1024, 832, 1),
                           seed=2, 
                           batch=batch_size, 
                           rtype='rect')

2023-04-30 11:28:19.941164: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-04-30 11:28:19.991760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 11:28:19.991980: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2023-04-30 11:28:19.991993: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-04-30 11:28:19.994022: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-04-30 11:28:19.994049: I tensorflow/stream_executor/pl

## Original UNet

### Elliptical ROI:

In [22]:
model_path = '/home/careinfolab/unet_mammo/base_unet/unet_original_test7/'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1024, 832, 64 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1024, 832, 64 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 512, 416, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [23]:
test_loss, test_dice, test_iou = model.evaluate(elli_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 51s 68ms/step - loss: -0.1867 - dice_coef: 0.2342 - iou_coef: 0.1550
Loss:  -0.18669822812080383
DICE:  0.23420929908752441
IOU:  0.15502415597438812


### Rectangular ROI:

In [27]:
model_path = '/home/careinfolab/unet_mammo/base_unet/unet_original_test6/'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1024, 832, 64 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1024, 832, 64 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 512, 416, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [28]:
test_loss, test_dice, test_iou = model.evaluate(rect_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 49s 71ms/step - loss: -0.1968 - dice_coef: 0.2485 - iou_coef: 0.1624
Loss:  -0.1968419998884201
DICE:  0.24846112728118896
IOU:  0.1624293178319931


### --------------------------------------------------------------------------------------------------------------------
## ResUNet

### Elliptical ROI:

In [29]:
model_path = './saved_models/resunet_test1_elli'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1024, 832, 32 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1024, 832, 32 128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1024, 832, 32 9248        batch_normalization[0][0]        
______________________________________________________________________________________________

In [30]:
test_loss, test_dice, test_iou = model.evaluate(elli_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 23s 33ms/step - loss: -0.1536 - dice_coef: 0.1928 - iou_coef: 0.1275
Loss:  -0.15362763404846191
DICE:  0.1928020715713501
IOU:  0.12751144170761108


### Rectangular ROI:

In [9]:
model_path = './saved_models/resunet_test1_rect'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 1024, 832, 32 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1024, 832, 32 128         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1024, 832, 32 9248        batch_normalization[0][0]        
______________________________________________________________________________________________

In [12]:
test_loss, test_dice, test_iou = model.evaluate(rect_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 25s 36ms/step - loss: -0.2057 - dice_coef: 0.2563 - iou_coef: 0.1719
Loss:  -0.20566609501838684
DICE:  0.2563225328922272
IOU:  0.17189522087574005


### --------------------------------------------------------------------------------------------------------------------
## BiSeNetV2

### Elliptical ROI:

In [14]:
model_path = './saved_models/bisenetv2_test1_elli'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
semantic_stem (SemanticStem)    (None, 256, 208, 16) 6312        input_1[0][0]                    
__________________________________________________________________________________________________
gather_expansion_stride2 (Gathe (None, 128, 104, 32) 21024       semantic_stem[0][0]              
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 512, 416, 64) 640         input_1[0][0]                    
______________________________________________________________________________________________

In [15]:
test_loss, test_dice, test_iou = model.evaluate(elli_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 10s 13ms/step - loss: -0.1707 - dice_coef: 0.2122 - iou_coef: 0.1430
Loss:  -0.17070819437503815
DICE:  0.21221531927585602
IOU:  0.14303676784038544


### Rectangular ROI:

In [16]:
model_path = './saved_models/bisenetv2_test1_rect'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
semantic_stem (SemanticStem)    (None, 256, 208, 16) 6312        input_1[0][0]                    
__________________________________________________________________________________________________
gather_expansion_stride2 (Gathe (None, 128, 104, 32) 21024       semantic_stem[0][0]              
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 512, 416, 64) 640         input_1[0][0]                    
______________________________________________________________________________________________

In [17]:
test_loss, test_dice, test_iou = model.evaluate(rect_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 10s 14ms/step - loss: -0.1922 - dice_coef: 0.2358 - iou_coef: 0.1631
Loss:  -0.19216419756412506
DICE:  0.23582330346107483
IOU:  0.1630580872297287


### --------------------------------------------------------------------------------------------------------------------
## DeepLabV3+ w/ SepResNet50V2

### Elliptical ROI:

In [18]:
model_path = './saved_models/deep_unet_test1_elli'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 416, 32) 1600        input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 208, 64) 2400        conv2d[0][0]                     
__________________________________________________________________________________________________
residual_block (ResidualBlock)  (None, 256, 208, 64) 14592       separable_conv2d[0][0]           
____________________________________________________________________________________________

In [19]:
test_loss, test_dice, test_iou = model.evaluate(elli_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 8s 11ms/step - loss: -0.1747 - dice_coef: 0.2224 - iou_coef: 0.1430
Loss:  -0.174730584025383
DICE:  0.22237753868103027
IOU:  0.142966166138649


### Rectangular ROI:

In [20]:
model_path = './saved_models/deep_unet_test1_rect2'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 416, 32) 1600        input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 208, 64) 2400        conv2d[0][0]                     
__________________________________________________________________________________________________
residual_block (ResidualBlock)  (None, 256, 208, 64) 14592       separable_conv2d[0][0]           
____________________________________________________________________________________________

In [21]:
test_loss, test_dice, test_iou = model.evaluate(rect_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 10s 14ms/step - loss: -0.1862 - dice_coef: 0.2351 - iou_coef: 0.1537
Loss:  -0.186221182346344
DICE:  0.2350640892982483
IOU:  0.15365950763225555


### --------------------------------------------------------------------------------------------------------------------
## DeepLabV3+ w/ SepResNet101V2

### Elliptical ROI:

In [31]:
model_path = './saved_models/deep_unet_test101_elli'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 416, 32) 1600        input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 208, 64) 2400        conv2d[0][0]                     
__________________________________________________________________________________________________
residual_block (ResidualBlock)  (None, 256, 208, 64) 14592       separable_conv2d[0][0]           
____________________________________________________________________________________________

In [32]:
test_loss, test_dice, test_iou = model.evaluate(elli_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 11s 15ms/step - loss: -0.1917 - dice_coef: 0.2384 - iou_coef: 0.1605
Loss:  -0.19165238738059998
DICE:  0.23839300870895386
IOU:  0.1604919582605362


### Rectangular ROI:

In [33]:
model_path = './saved_models/deep_unet_test101_rect2'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 416, 32) 1600        input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 208, 64) 2400        conv2d[0][0]                     
__________________________________________________________________________________________________
residual_block (ResidualBlock)  (None, 256, 208, 64) 14592       separable_conv2d[0][0]           
____________________________________________________________________________________________

In [34]:
test_loss, test_dice, test_iou = model.evaluate(rect_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 12s 17ms/step - loss: -0.2077 - dice_coef: 0.2590 - iou_coef: 0.1736
Loss:  -0.20772013068199158
DICE:  0.2589723765850067
IOU:  0.1735517978668213


### --------------------------------------------------------------------------------------------------------------------
## DeepLabV3+ w/ SepResNet152V2

### Elliptical ROI:

In [9]:
model_path = './saved_models/deep_unet_test152_elli1'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 416, 32) 1600        input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 208, 64) 2400        conv2d[0][0]                     
__________________________________________________________________________________________________
residual_block (ResidualBlock)  (None, 256, 208, 64) 14592       separable_conv2d[0][0]           
____________________________________________________________________________________________

In [8]:
test_loss, test_dice, test_iou = model.evaluate(elli_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

688/688 [==============================] - 14s 19ms/step - loss: -0.1978 - dice_coef: 0.2450 - iou_coef: 0.1663
Loss:  -0.197812020778656
DICE:  0.2450248748064041
IOU:  0.16633670032024384


### Rectangular ROI:

In [5]:
model_path = './saved_models/deep_unet_test152_rect1'

model = load_model(model_path, compile=False)
model.compile(loss=[seg_loss], metrics=[dice_coef, iou_coef])
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 832, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 416, 32) 1600        input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 208, 64) 2400        conv2d[0][0]                     
__________________________________________________________________________________________________
residual_block (ResidualBlock)  (None, 256, 208, 64) 14592       separable_conv2d[0][0]           
____________________________________________________________________________________________

In [6]:
test_loss, test_dice, test_iou = model.evaluate(rect_test_df)
print('Loss: ', test_loss)
print('DICE: ', test_dice)
print('IOU: ', test_iou)

2023-04-30 11:29:36.060420: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-04-30 11:29:36.076956: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3800060000 Hz
2023-04-30 11:29:37.189509: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-04-30 11:29:37.643528: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2023-04-30 11:29:38.222339: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


      3/Unknown - 3s 26ms/step - loss: -0.1863 - dice_coef: 0.2496 - iou_coef: 0.1442

2023-04-30 11:29:38.623336: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


688/688 [==============================] - 17s 21ms/step - loss: -0.2010 - dice_coef: 0.2510 - iou_coef: 0.1677
Loss:  -0.20104730129241943
DICE:  0.25101780891418457
IOU:  0.16773371398448944
